In [10]:
import json
import numpy as np
from collections import defaultdict
import customutils

def getIDByName(labels, name):
    return labels.index(name)

def get_keypointconstants(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [float(c.strip()) for c in class_names]
    return class_names

def getHeadSize(x1,y1,x2,y2):
    headSize = 0.6*np.linalg.norm(np.subtract([x2,y2],[x1,y1]))
    return headSize

def getTopPoint(points):
    point = []
    for i in range(len(points)):
        if (points[i]["id"] is not None and points[i]["id"] == "Nose"): # if joint id matches
            point = points[i]
            break

    return point

def getPointbyID(points,id):

    point = []
    for i in range(len(points)):
        if (points[i]["id"] is not None and points[i]["id"] == id): # if joint id matches
            point = points[i]
            break

    return point

In [11]:
KEYPOINT_CONSTANTS = get_keypointconstants('configs/keypointconstant.txt')
LABELS_COCO = customutils.get_classes('configs/coco_classes.txt')


def compute_scale(image_size, bounding_box):
    # Get the width and height of the image
    image_width, image_height = image_size

    # Calculate the width and height of the bounding box
    box_width = bounding_box[2][0] - bounding_box[0][0]
    box_height = bounding_box[2][1] - bounding_box[0][1]

    # Calculate the scale factors
    scale_width = box_width / image_width
    scale_height = box_height / image_height

    # Choose the minimum scale factor
    scale = min(scale_width, scale_height)

    return scale

def compute_oks(joint, d, area, visibility):
    k = KEYPOINT_CONSTANTS[joint]
    
    # Compute the exponential part of the equation
    exp_vector = np.exp(-(d**2) / (2 * (area) * (k**2)))
    # The numerator expression
    numerator = np.dot(exp_vector, visibility)
    # The denominator expression
    denominator = np.sum(visibility)
    return numerator / denominator

In [12]:
import numpy as np

from decimal import Decimal, ROUND_HALF_UP


def edit_keypoints(kpts):
    kpts = np.array(kpts).reshape(-1,3)
    vi = kpts[:,2]
    kpts = kpts[:,0:2]
    return kpts, vi


def OKS(kpts1, kpts2, sigma, area):

    kpts1, vi1 = edit_keypoints(kpts1)
    kpts2, vi2 = edit_keypoints(kpts2)

    if np.shape(kpts1) != np.shape(kpts2):
        print(kpts1, kpts2)
        print(np.shape(kpts1), np.shape(kpts2))
        raise ValueError("not same size")
    
    k = 2*sigma
    s = area

    d = np.linalg.norm(kpts1 - kpts2, ord=2, axis=1)
    v = np.ones(len(d))

    for part in range(len(d)):
        if vi1[part] == 0 or vi2[part] == 0:
            d[part] = 0
            v[part] = 0
    
    if np.sum(v)!=0:
        OKS = (np.sum([(np.exp((-d[i]**2)/(2*s*(k[i]**2))))*v[i] for i in range(len(d))])/np.sum(v))
    else:
        OKS = 0
    
    OKS = float(Decimal(str(OKS)).quantize(Decimal('0.000001'), rounding=ROUND_HALF_UP))

    return OKS

In [13]:
def computeDistOks(gtLabels, gtFrames, prFrames):
    # assert len(gtFrames) == len(prFrames)

    nJoints = len(gtLabels)
    distAll = {}
    dist = {}
    ious = {}
    iousAll = {}
    oksAll = defaultdict(list)
    for joint in range(nJoints):
        distAll[joint] = np.zeros([0,0])
        iousAll[joint] = []

    # for imgidx in range(len(gtFrames)):
    for imgidx, gtFrame in gtFrames.items():
        oksAll[imgidx] = []
        # ground truth
        # gtFrame = gtFrames[imgidx]
        # prediction
        detFrame = prFrames[imgidx]
        prFramesLen = len(detFrame)
        dist = np.ones((len(gtFrame), max(prFramesLen,1), nJoints)) * np.inf
        ious = np.zeros((len(gtFrame), max(prFramesLen,1), nJoints))
        oks_all = np.zeros((len(gtFrame), max(prFramesLen,1)))
        
        for personGT in range(len(gtFrame)):
            rectGT = gtFrames[imgidx][personGT]
            # if no poses predicted, initialise
            if prFramesLen == 0:
                for pidx in range(nJoints):
                    dist[personGT, 0, pidx] = np.inf
            else:
                # compute distance between each detection and ground truth object
                for personDT in range(prFramesLen):
                    rectPr = prFrames[imgidx][personDT]
                    if ("person" in rectGT.keys() and rectGT["person"] is not None):
                        pointsGT = rectGT["person"]["points"]
                        pointsPr = rectPr["person"]["points"]
                        bb = customutils.get_bb(pointsGT)
                        area = customutils.compute_area(bb)

                        for pidx, idxGT in enumerate(gtLabels):
                            keypointGT = getPointbyID(pointsGT,idxGT)
                            pointGT = [keypointGT["x"],keypointGT["y"]]
                            idxGT = keypointGT["id"]
                            #idx = getIDByName(gtLabels, idxGT)
                            p = getPointbyID(pointsPr,idxGT)
                            if (len(p) > 0 and
                                isinstance(p["x"], (int, float)) and
                                isinstance(p["y"], (int, float)) and 
                                p["x"] > 0 and
                                p["y"] > 0):
                                pointPr = [p["x"],p["y"]]
                                visible = 1
                                if "occluded" in keypointGT.keys() and keypointGT["occluded"] is not None:
                                    if keypointGT["occluded"] == "False":
                                        visible = 2
                                else:
                                    visible = keypointGT["visibility"]
                                # compute distance between GT and prediction
                                d = np.linalg.norm(np.subtract(pointGT,pointPr))
                                oks = compute_oks(pidx, d, area, visible)
                                # compute head size for distance normalization
                                head = getPointbyID(pointsGT,"Head")
                                neck = getPointbyID(pointsGT,"Neck")
                                headSize = 1
                                if (len(head) > 0 and len(neck) > 0):
                                    headSize = getHeadSize(head["x"], head["y"], neck["x"], neck["y"])
                                # normalize distance
                                dNorm = d/headSize
                            else:
                                dNorm = np.inf
                                oks = 0

                            dist[personGT, personDT, pidx] = dNorm
                            ious[personGT, personDT, pidx] = oks
                            
                        oks_all[personGT, personDT] = OKS(rectGT["person"]["kpts"], rectPr["person"]["kpts"], KEYPOINT_CONSTANTS, area)


        #find matching detection (minimum distance) for each ground truth
        for personGT in range(len(gtFrames[imgidx])):
            min_dist = np.inf
            if prFramesLen == 0:
                minPersonDT = 0
            for personDT in range(len(prFrames[imgidx])):
                total_dist = 0
                for idx in range(nJoints):
                    if dist[personGT, personDT, idx] != np.inf:
                        total_dist += dist[personGT, personDT, idx]
                if total_dist < min_dist:
                    min_dist = total_dist
                    minPersonDT = personDT
        
            for idx in range(nJoints):
                #print(str(dist[personGT, minPersonDT, idx]) + ' ' + str(personGT) + ' ' + str(minPersonDT) + ' ' + str(idx))
                distAll[idx] = np.append(distAll[idx],[[dist[personGT, minPersonDT, idx]]])
                iousAll[idx].append(ious[personGT, minPersonDT, idx])

            oksAll[imgidx].append(oks_all[personGT, minPersonDT])


    return distAll, iousAll, oksAll

def computeAP(labels, iousAll, thresh):

    ap = np.zeros([len(iousAll)+2,1])
    nCorrect = 0
    nTotal = 0
    nCorrectCoco = 0
    nTotalCoco = 0
    for pidx in range(len(iousAll)):
        if len(iousAll[pidx]) == 0:
            print (pidx)
            ap[pidx,0] = np.inf
        else:
            threshArr = np.ones(len(iousAll[pidx]))*thresh
            idxs = np.argwhere(iousAll[pidx] >= threshArr)
            oks = 100.0*len(idxs)/len(iousAll[pidx])
            ap[pidx,0] = oks
            nCorrect += len(idxs)
            nTotal   += len(iousAll[pidx])
            if labels[pidx] in LABELS_COCO:
                nCorrectCoco += len(idxs)
                nTotalCoco   += len(iousAll[pidx])
    ap[len(iousAll),0] = 100.0*nCorrect/nTotal
    ap[len(iousAll)+1,0] = 100.0*nCorrectCoco/nTotalCoco

    return ap

def computeAP_OKS(labels, iousAll, thresh):

    ap = np.zeros([len(iousAll)+2,1])
    nCorrect = 0
    nTotal = 0
    idx = 0
    for pidx, val in iousAll.items():
        threshArr = np.ones(len(iousAll[pidx]))*thresh
        idxs = np.argwhere(iousAll[pidx] >= threshArr)
        oks = 100.0*len(idxs)/len(iousAll[pidx])
        ap[idx,0] = oks
        nCorrect += len(idxs)
        nTotal   += len(iousAll[pidx])
        idx += 1
    ap[len(iousAll),0] = 100.0*nCorrect/nTotal

    return ap

def computePCK(labels, distAll, distThresh):

    pckAll = np.zeros([len(distAll)+2,1])
    nCorrect = 0
    nTotal = 0
    nCorrectCoco = 0
    nTotalCoco = 0
    for pidx in range(len(distAll)):
        idxs = np.argwhere(distAll[pidx] <= distThresh)
        pck = 100.0*len(idxs)/len(distAll[pidx])
        pckAll[pidx,0] = pck
        nCorrect += len(idxs)
        nTotal   += len(distAll[pidx])
        if labels[pidx] in LABELS_COCO:
            nCorrectCoco += len(idxs)
            nTotalCoco   += len(distAll[pidx])
    pckAll[len(distAll),0] = 100.0*nCorrect/nTotal
    pckAll[len(distAll)+1,0] = 100.0*nCorrectCoco/nTotalCoco

    return pckAll


def computeMetrics(gtLabels, names, gtFramesAll, prFramesAll, file):

    distThresh = 0.5

    # compute distances
    distAll, oks, oksAll = computeDistOks(gtLabels, gtFramesAll, prFramesAll)

    ap50 = computeAP(gtLabels, oks, distThresh)
    ap = np.copy(ap50)
    for i in range(9):
        distThresh += 0.05
        ap += computeAP(gtLabels, oks, distThresh)
    ap75 = computeAP(gtLabels, oks, 0.75)
    ap = ap/10

    ap50_2 = computeAP_OKS(gtLabels, oksAll, distThresh)
    ap_2 = np.copy(ap50_2)
    for i in range(9):
        distThresh += 0.05
        ap_2 += computeAP_OKS(gtLabels, oksAll, distThresh)
    ap75_2 = computeAP_OKS(gtLabels, oksAll, 0.75)
    ap_2 = ap_2/10

    # compute PCK metric
    pckAll = computePCK(gtLabels, distAll, distThresh)
    metrics = {'pckh': pckAll.flatten().tolist(), 
               'AP': ap.flatten().tolist(),  
               'AP0.5': ap50.flatten().tolist(),  
               'AP0.75': ap75.flatten().tolist(),  
               'AP_2': ap_2.flatten().tolist(),  
               'AP0.5_2': ap50_2.flatten().tolist(),  
               'AP0.75_2': ap75_2.flatten().tolist(),  
               'names': names}
    filename = 'results/results_pckh_' + file + '.json'
    print('saving results to', filename)
    customutils.writeJson(metrics,filename)

    return pckAll, ap, ap50, ap75#, ap_2, ap50_2, ap75_2


#load_data("test.json")

In [14]:
import os, argparse, json, re

def to60fps(i):
    return i*2

def load_prediction_files(gtLabels, labels, file_path, frame_count):
    frames = defaultdict(list)
    common_labels = set()

    for filename in os.listdir(file_path):
        if filename.endswith('.json'):
            full_file = os.path.join(file_path, filename)
            with open(full_file, 'r') as f:            
                pattern = '([0-9]*)([_a-z]*)\.json$'
                result = re.search(pattern, filename)
                frame_index = int(result.groups(0)[0])
                if frame_index < frame_count:
                    predictions_annotations = json.load(f)
                    person_id = 0
                    for pose in predictions_annotations["people"]:
                        keypoints = frames[frame_index]
                        pose_kpts = pose["pose_keypoints_2d"]
                        keypoints_index = 0
                        points = []
                        confidences = []
                        kpts = []
                        for label in labels:
                            if label in gtLabels:
                                common_labels.add(label)
                                if keypoints_index < len(pose_kpts):
                                    points.append({'id': label, 'x': pose_kpts[keypoints_index], 'y': pose_kpts[keypoints_index+1], 'confidence': pose_kpts[keypoints_index+2]})
                                    kpts.append(pose_kpts[keypoints_index])
                                    kpts.append(pose_kpts[keypoints_index+1])
                                    kpts.append(2) #visibility
                                    confidences.append(pose_kpts[keypoints_index+2])
                            keypoints_index = keypoints_index + 3

                        keypoint = {'points': points, 'confidence': confidences, 'kpts' : kpts}

                        keypoints.append({'person': keypoint, 'person_id': person_id})
                        person_id += 1

                        frames[frame_index] = keypoints


    return frames, common_labels


def load_alphaposeframe(pose, frames, frame_count, prev_frameindex, person_id, labels, gtLabels, keypoints):
    if ("image_id" in pose and pose["image_id"] is not None):
            frame_id = pose["image_id"]
    else:
        frame_id = pose
    frame_index = int(frame_id.replace(".jpg", ""))
    if prev_frameindex == frame_index:
        person_id += 1
    else:
        person_id = 0
    if frame_index < frame_count:
        keypoints = frames[frame_index]
        pose_kpts = pose["keypoints"]
        keypoints_index = 0
        points = []
        confidences = []
        kpts = []
        for label in labels:
            if keypoints_index < len(pose_kpts):
                points.append({'id': label, 'x': pose_kpts[keypoints_index], 'y': pose_kpts[keypoints_index+1], 'confidence': pose_kpts[keypoints_index+2]})
                confidences.append(pose_kpts[keypoints_index+2])
                if label in gtLabels:
                    kpts.append(pose_kpts[keypoints_index])
                    kpts.append(pose_kpts[keypoints_index+1])
                    kpts.append(2) #visibility
                    
                keypoints_index = keypoints_index + 3

        keypoint = {'points': points, 'confidence': confidences, 'kpts' : kpts}

        keypoints.append({'person': keypoint, 'person_id': person_id })

        frames[frame_index] = keypoints
    prev_frameindex = frame_index
    return frames, prev_frameindex, person_id, keypoints

def load_predictions(gtLabels, labels, file_path, frame_count):
  frames = defaultdict(list)
  common_labels = set()

  with open(file_path, 'r') as f:
      predictions_annotations = json.load(f)

  keypoints = []
  if ("instance_info" in predictions_annotations and predictions_annotations["instance_info"] is not None):
     for frame in predictions_annotations["instance_info"]:
        frame_index = frame["frame_id"]-1
        if frame_index < frame_count:
            person_id = 0
            keypoints = frames[frame_index]
            for pose in frame["instances"]:
                pose_kpts = pose["keypoints"]
                keypoints_index = 0
                points = []
                confidences = []
                kpts = []
                for i, idx in enumerate(labels):
                    if idx in gtLabels:
                        common_labels.add(idx)
                    if i < len(pose_kpts):
                        points.append({'id': idx, 'x': pose_kpts[i][0], 'y': pose_kpts[i][1], 'confidence': pose["keypoint_scores"][i]})
                        confidences.append(pose["keypoint_scores"][i])
                        kpts.append(pose_kpts[i][0])
                        kpts.append(pose_kpts[i][1])
                        kpts.append(2) #visibility

                keypoint = {'points': points, 'confidence': confidences, 'kpts' : kpts}

                keypoints.append({'person': keypoint, 'person_id': person_id})
                person_id += 1

            frames[frame_index] = keypoints
  else:
    prev_frameindex = -1
    person_id = 0
    for pose in predictions_annotations:
        frames, prev_frameindex, person_id, keypoints = load_alphaposeframe(pose, frames, frame_count, prev_frameindex, person_id, labels, gtLabels, keypoints)

  return frames, common_labels


def load_annotations(labels, gtLabels, file_path):
    frames = defaultdict(list)
    isExist = os.path.exists(file_path)
    keypoints = defaultdict(list)
    if isExist:
      annotations = []
      with open(file_path, 'r') as f:
          annotations = json.load(f)

          framecount = annotations['item']['slots'][0]['frame_count']
          for person in annotations['annotations']:
            
            for frame_index in range(0, framecount):
              points = []
              if frame_index < len(person['frames']):
                frame = person['frames'][str(frame_index)]
                for node in frame['skeleton']['nodes']:
                  if node['name'] in gtLabels:
                    points.append({'id': node['name'], 'occluded' : node['occluded'], 'x' : node['x'], 'y': node['y']})

              if len(points) > 0:
                # Create a dictionary to map 'id' to its position in the order array
                id_to_index = {id_value: index for index, id_value in enumerate(labels)}

                # Sort the data based on the custom sorting key
                sorted_data = sorted(points, key=lambda x: id_to_index.get(x['id'], float('inf')))
                kpts = []
                for point in sorted_data:
                    if point['id'] in labels: #check joint is available for this technique
                        kpts.append(point['x'])
                        kpts.append(point['y'])

                        if point["occluded"] == "false":
                            kpts.append(2)
                        else:                       
                            kpts.append(1)

                index = frame_index
                keypoints[index].append({'person': {'points': points, 'kpts' : kpts}})

        #   for frame in range(0, len(keypoints)):
        #     frames[frame] = keypoints[frame]

    return keypoints

def load_data(file):
    with open(file, 'r') as f:
      manifest = json.load(f)

    gtLabels = customutils.get_classes('configs/halpe26_classes.txt')
    
    names = gtLabels[:]
    names.append('Total')
    names.append('Total (COCO)')

    apAll = np.zeros([len(names),1])
    ap50All = np.zeros([len(names),1])
    ap75All = np.zeros([len(names),1])
    pckAll = np.zeros([len(names),1])

    for i in manifest['index']:
        print('Processing ' + i['file'])

        labels = customutils.get_classes(manifest['labels'])
        gt = load_annotations(labels, gtLabels, i['annotations'])
        predictions = i['predictions']
        if os.path.isfile(predictions):
            pred, labels = load_predictions(gtLabels, labels, predictions, len(gt))
        else:
            pred, labels = load_prediction_files(gtLabels, labels, predictions, len(gt))

        pck, ap, ap50, ap75 = computeMetrics(gtLabels, names, gt, pred, manifest['model'] + '_' + i['name'])
        pckAll = (pckAll + pck)
        apAll = (apAll + ap)
        ap50All = (ap50All + ap50)
        ap75All = (ap75All + ap75)

    pckAll = pckAll/len(manifest['index'])
    apAll = apAll/len(manifest['index'])
    ap50All = ap50All/len(manifest['index'])
    ap75All = ap75All/len(manifest['index'])

    metrics = {'pckh': pckAll.flatten().tolist(), 
               'AP': apAll.flatten().tolist(),  
               'AP0.5': ap50All.flatten().tolist(),  
               'AP0.75': ap75All.flatten().tolist(), 
               'names': names}
    filename = 'results/results_' +  file #manifest['model'] + '.json'
    print('saving results to', filename)
    customutils.writeJson(metrics,filename)

In [15]:

def load_keypoints(gtLabels, predLabels, gt, annotations):
    keypoints_index = 0
    points = []
    confidences = []
    pose_kpts = annotations['keypoints']
    for label in gtLabels:
      points.append({'id': label, 'x': pose_kpts[keypoints_index], 'y': pose_kpts[keypoints_index+1], 'visibility': pose_kpts[keypoints_index+2]})
      keypoints_index = keypoints_index + 3

    # Create a dictionary to map 'id' to its position in the order array
    id_to_index = {id_value: index for index, id_value in enumerate(predLabels)}

    # Sort the data based on the custom sorting key
    sorted_data = sorted(points, key=lambda x: id_to_index.get(x['id'], float('inf')))
    kpts = []
    for point in sorted_data:
      kpts.append(point['x'])
      kpts.append(point['y'])
      kpts.append(point['visibility'])

    gt[annotations['image_id']].append({'person': {'points': sorted_data, 'kpts' : kpts}})

    return gt


def load_alphapose_predictions(gtLabels, labels, file_path, frame_count):
  frames = defaultdict(list)
  common_labels = set()

  with open(file_path, 'r') as f:
    pose = json.load(f)

    prev_frameindex = -1
    person_id = 0
    keypoints = []
    frames, prev_frameindex, person_id, keypoints = load_alphaposeframe(pose, frames, frame_count, prev_frameindex, person_id, labels, gtLabels, keypoints)

  return frames, common_labels

def load_mpii():
    with open('../data/mpii/trainval.json', 'r') as f:
      manifest = json.load(f)

def load_coco(model):
    with open('../data/coco/annotations/person_keypoints_val2017.json', 'r') as f:
      manifest = json.load(f)

    gtLabels = customutils.get_classes('configs/coco_classes.txt')
    
    names = gtLabels[:]
    names.append('Total')
    names.append('Total (COCO)')

    apAll = np.zeros([len(names),1])
    ap50All = np.zeros([len(names),1])
    ap75All = np.zeros([len(names),1])
    pckAll = np.zeros([len(names),1])

    gt = defaultdict(list)
    predictions = defaultdict(list)
    image_count = 0
    for i in manifest['annotations']:
      image_name = str(i['image_id']).zfill(12)
      image = image_name + '.jpg'
      image_count += 1

      if model == 'alphapose':
        labels = customutils.get_classes('configs/halpe26_classes.txt')

        gt = load_keypoints(gtLabels, labels, gt, i)
        predictions_file = '../data/output/alphapose/coco/val2017/' + image_name + '/alphapose-results.json'
        # pred = load_alphapose_predictions(gtLabels, labels, file_path, 1)
        if os.path.isfile(predictions_file):
          pred, labels = load_predictions(gtLabels, labels, predictions_file, np.inf)
        else:
          pred, labels = load_prediction_files(gtLabels, labels, predictions_file, np.inf)

        for k,v in pred.items():
           predictions[k] = v

    pck, ap, ap50, ap75 = computeMetrics(gtLabels, names, gt, predictions, model + '_' + image_name)
    pckAll = (pckAll + pck)
    apAll = (apAll + ap)
    ap50All = (ap50All + ap50)
    ap75All = (ap75All + ap75)

    pckAll = pckAll/image_count
    apAll = apAll/image_count
    ap50All = ap50All/image_count
    ap75All = ap75All/image_count
        
    metrics = {'pckh': pckAll.flatten().tolist(), 
               'AP': apAll.flatten().tolist(),  
               'AP0.5': ap50All.flatten().tolist(),  
               'AP0.75': ap75All.flatten().tolist(),  
               'AP_2': ap_2.flatten().tolist(),  
               'AP0.5_2': ap50_2.flatten().tolist(),  
               'AP0.75_2': ap75_2.flatten().tolist(), 
               'names': names}
    filename = 'results/results_coco_' +  model + '.json'
    print('saving results to', filename)
    customutils.writeJson(metrics,filename)

In [16]:

def load_cotracker_pose(pose, frames, frame_index, labels, gtLabels, keypoints):
    keypoints = frames[frame_index]
    pose_kpts = pose["keypoints"]
    keypoints_index = 0
    points = []
    confidences = []
    kpts = []
    for label in labels:
        if keypoints_index < len(pose_kpts):
            points.append({'id': label, 'x': pose_kpts[keypoints_index], 'y': pose_kpts[keypoints_index+1], 'confidence': pose_kpts[keypoints_index+2]})
            confidences.append(pose_kpts[keypoints_index+2])
            if label in gtLabels:
                kpts.append(pose_kpts[keypoints_index])
                kpts.append(pose_kpts[keypoints_index+1])
                kpts.append(2) #visibility
                
            keypoints_index = keypoints_index + 3

    keypoint = {'points': points, 'confidence': confidences, 'kpts' : kpts}

    keypoints.append({'person': keypoint })

    frames[frame_index] = keypoints
        
    return frames, keypoints

def load_data_cotracker(path):
    gtLabels = customutils.get_classes('configs/halpe26_classes.txt')
    
    names = gtLabels[:]
    names.append('Total')
    names.append('Total (COCO)')

    apAll = np.zeros([len(names),1])
    ap50All = np.zeros([len(names),1])
    ap75All = np.zeros([len(names),1])
    pckAll = np.zeros([len(names),1])

    for filename in os.listdir(path):
        if filename.endswith('.json'):
            file_path = os.path.join(path, filename)
            with open(file_path, 'r') as f:
                pred_cotracker = json.load(f)
                frame_index = 0
                keypoints = []
                frames = defaultdict(list)
                for pose in pred_cotracker['frames']:
                    frames, keypoints = load_cotracker_pose(pose, frames, frame_index, gtLabels, gtLabels, keypoints)
                    frame_index += 1
            print('Processing ' + filename)

            gt = load_annotations(gtLabels, gtLabels, '../videos/annotations/' + filename)

            pck, ap, ap50, ap75 = computeMetrics(gtLabels, names, gt, frames, 'cotracker_' + filename)
            pckAll = (pckAll + pck)
            apAll = (apAll + ap)
            ap50All = (ap50All + ap50)
            ap75All = (ap75All + ap75)

    pckAll = pckAll/len(pckAll)
    apAll = apAll/len(apAll)
    ap50All = ap50All/len(ap50All)
    ap75All = ap75All/len(ap75All)

    metrics = {'pckh': pckAll.flatten().tolist(), 
               'AP': apAll.flatten().tolist(),  
               'AP0.5': ap50All.flatten().tolist(),  
               'AP0.75': ap75All.flatten().tolist(), 
               'names': names}
    filename = 'results/results_cotracker.json'
    print('saving results to', filename)
    customutils.writeJson(metrics,filename)

In [8]:
load_coco('alphapose')

In [19]:
load_data_cotracker("../videos/results/cotracker")

Processing Athletics_Mixed_Tokyo_2020_20_1.json
saving results to results/results_pckh_cotracker_Athletics_Mixed_Tokyo_2020_20_1.json.json
Processing Athletics_Mixed_Tokyo_2020_39.json
saving results to results/results_pckh_cotracker_Athletics_Mixed_Tokyo_2020_39.json.json
Processing Athletics_Mixed_Tokyo_2020_64.json
saving results to results/results_pckh_cotracker_Athletics_Mixed_Tokyo_2020_64.json.json
saving results to results/results_cotracker.json


In [22]:
load_data("manifest_vitpose.json")
load_data("manifest_openpose.json")
load_data("manifest_alphapose.json")

Processing Athletics_Mixed_Tokyo_2020_20_1.avi
saving results to results/results_pckh_vitpose_Athletics_Mixed_Tokyo_2020_20_1.json
Processing Triathlon_Women_Tokyo_2020_29.avi
saving results to results/results_pckh_vitpose_Triathlon_Women_Tokyo_2020_29.json
Processing World_Athletics_Men_10000m_Oregon_2022_1.mp4
saving results to results/results_pckh_vitpose_World_Athletics_Men_10000m_Oregon_2022_1.json
Processing World_Athletics_Men_10000m_Oregon_2022_2.mp4
saving results to results/results_pckh_vitpose_World_Athletics_Men_10000m_Oregon_2022_2.json
Processing World_Athletics_Men_10000m_Oregon_2022_23.mp4
saving results to results/results_pckh_vitpose_World_Athletics_Men_10000m_Oregon_2022_23.json
Processing World_Athletics_Men_10000m_Oregon_2022_25.mp4
saving results to results/results_pckh_vitpose_World_Athletics_Men_10000m_Oregon_2022_25.json
Processing World_Athletics_Men_10000m_Oregon_2022_28.mp4
saving results to results/results_pckh_vitpose_World_Athletics_Men_10000m_Oregon_2022

In [9]:

load_data("vitpose.json")
load_data("rtmpose_cocowb.json")
load_data("openpose.json")
load_data("alphapose_halpe.json")

Processing clip_tri_1.mp4
saving results to results/results_pckh_openpose_clip_tri_1.json
Processing clip_tri_3.mp4
saving results to results/results_pckh_openpose_clip_tri_3.json
Processing clip_marathon_1.mp4
saving results to results/results_pckh_openpose_clip_marathon_1.json
Processing clip_10k_2.mp4
saving results to results/results_pckh_openpose_clip_10k_2.json
Processing short.mp4
saving results to results/results_pckh_openpose_short.json
saving results to results/results_openpose.json


In [14]:

def load_images_annotations(labels, path):
    keypoints = defaultdict(list)
    for filename in os.listdir(path):
        if filename.endswith('.json'):
            file_path = os.path.join(path, filename)
            kpts = []
            with open(file_path, 'r') as f:
                annotations = json.load(f)

                for person in annotations['annotations']:
                    points = person['skeleton']['nodes']
                    # Create a dictionary to map 'id' to its position in the order array
                    id_to_index = {id_value: index for index, id_value in enumerate(labels)}

                    # Sort the data based on the custom sorting key
                    sorted_data = sorted(points, key=lambda x: id_to_index.get(x['name'], float('inf')))
                    
                    for point in sorted_data:
                        if point['name'] in labels: #check joint is available for this technique
                            kpts.append(point['x'])
                            kpts.append(point['y'])

                            if point["occluded"] == False:
                                kpts.append(2)
                            else:                       
                                kpts.append(1)
                
                    keypoints[filename] = kpts

    return keypoints

In [18]:
gtLabels = customutils.get_classes('configs/halpe26_classes.txt')

keypoints1 = load_images_annotations(gtLabels, '../data/coco/annotations/coco-gt')
keypoints2 = load_images_annotations(gtLabels, '../data/coco/annotations/coco-s')
keypoints3 = load_images_annotations(gtLabels, '../data/coco/annotations/coco-mr')

In [29]:
keypoints = defaultdict(list)
sigmas = defaultdict(list)

for k,v in keypoints1.items():
    kpts1, vi1 = customutils.edit_keypoints(keypoints1[k])
    kpts2, vi2 = customutils.edit_keypoints(keypoints2[k])
    kpts3, vi3 = customutils.edit_keypoints(keypoints3[k])

    # Calculate the average to get gt point (centroid)
    kpts = []
    for part in range(len(kpts1)):
        kpts_sum = 0
        kpts_cnt = 0
        if vi1[part] == 2:
            kpts_sum += kpts1[part]
            kpts_cnt += 1
        if vi2[part] == 2:
            kpts_sum += kpts2[part]
            kpts_cnt += 1
        if vi3[part] == 2:
            kpts_sum += kpts3[part]
            kpts_cnt += 1

        if kpts_cnt == 0:
            kpts_sum = [float(0), float(0)]
            kpts.append(list(kpts_sum))
        else:
            kpts.append(list(kpts_sum/kpts_cnt))

    kpts = np.array(kpts)
    s = customutils.compute_area_x_y_kpts(kpts[:,0], kpts[:,1])

    total_d = np.zeros(26)

    d = np.linalg.norm(kpts - kpts1, ord=2, axis=1)
    v = np.ones(len(kpts))
    for part in range(len(d)):
        if vi1[part] != 2:
            d[part] = 0
            v[part] = 0
    total_d += d

    d = np.linalg.norm(kpts - kpts2, ord=2, axis=1)
    v = np.ones(len(kpts))
    for part in range(len(d)):
        if vi2[part] != 2:
            d[part] = 0
            v[part] = 0
    total_d += d

    d = np.linalg.norm(kpts - kpts3, ord=2, axis=1)
    v = np.ones(len(kpts))
    for part in range(len(d)):
        if vi3[part] != 2:
            d[part] = 0
            v[part] = 0
    total_d += d

    # Calculate average keypoint distances
    d = total_d/3

    sigmas[k] = (d**2)/(s**2)

    #sigma = (np.mean([(d[i]**2)/(s**2) for i in range(len(d))])/np.sum(v))
    
sigma = (np.mean([sigmas[k] for k, v in sigmas.items()]))



In [31]:
sigmas

defaultdict(list,
            {'000000000785.json': array([7.84946167e-10, 7.23218381e-10, 1.10799621e-10, 3.68002285e-10,
                    0.00000000e+00, 9.78128507e-09, 3.69336956e-09, 2.26788007e-08,
                    7.18659982e-09, 6.44467630e-10, 1.83628513e-09, 6.56347340e-09,
                    6.65698969e-09, 6.81781160e-09, 7.24259033e-09, 2.01344290e-09,
                    3.25110523e-09, 5.71132251e-09, 3.98996863e-11, 2.24565378e-09,
                    7.17297009e-09, 6.38827050e-09, 2.31352023e-09, 4.30120153e-10,
                    1.52207976e-08, 1.55840091e-08]),
             '000000000872.json': array([6.32740998e-11, 5.52665514e-11, 0.00000000e+00, 7.76997498e-10,
                    0.00000000e+00, 1.05295776e-09, 7.99869928e-11, 6.33460420e-10,
                    0.00000000e+00, 5.64367151e-10, 8.54026054e-10, 6.26842240e-09,
                    2.78694602e-09, 1.67137376e-10, 6.28956195e-10, 1.19334501e-09,
                    8.51948900e-10, 3.48056486

In [27]:
sigma = 0

d = np.linalg.norm(kpts - kpts1, ord=2, axis=1)
v = np.ones(len(kpts))
for part in range(len(d)):
    if vi1[part] != 2:
        d[part] = 0
        v[part] = 0

sigma += (d**2)/(s**2)

d = np.linalg.norm(kpts - kpts2, ord=2, axis=1)
v = np.ones(len(kpts))
for part in range(len(d)):
    if vi2[part] != 2:
        d[part] = 0
        v[part] = 0

sigma += (d**2)/(s**2)

d = np.linalg.norm(kpts - kpts3, ord=2, axis=1)
v = np.ones(len(kpts))
for part in range(len(d)):
    if vi3[part] != 2:
        d[part] = 0
        v[part] = 0

sigma += (d**2)/(s**2)

sigma = sigma/3
np.sqrt(sigma)

array([4.92496481e-10, 4.64064167e-10, 1.87964404e-10, 3.35567027e-10,
       0.00000000e+00, 1.76083142e-09, 1.06722582e-09, 2.94758083e-09,
       1.46449939e-09, 5.30376860e-10, 7.76660553e-10, 1.50745785e-09,
       1.47076475e-09, 1.44120664e-09, 1.62668371e-09, 8.24421805e-10,
       1.06319344e-09, 1.40982428e-09, 1.31968036e-10, 8.63110715e-10,
       1.47596864e-09, 1.38274299e-09, 8.20965594e-10, 3.94653249e-10,
       2.18660744e-09, 2.23598619e-09])

In [14]:

kpts_d=[]
for part in range(len(kpts)):
    d = 0
    kpts_cnt = 0
    if vi1[part] == 2:
        d += kpts[part] - kpts1[part]
        kpts_cnt += 1
    if vi2[part] == 2:
        d += kpts[part] - kpts2[part]
        kpts_cnt += 1
    if vi3[part] == 2:
        d += kpts[part] - kpts3[part]
        kpts_cnt += 1

    kpts_d.append(list(d/kpts_cnt))
kpts_d = np.array(kpts_d)
kpts_d

array([[ 1.89478063e-14, -4.73695157e-15],
       [-3.78956126e-14, -4.73695157e-15],
       [-1.89478063e-14,  0.00000000e+00],
       [-1.89478063e-14,  4.73695157e-15],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.89478063e-14],
       [-1.89478063e-14,  4.73695157e-15],
       [ 3.78956126e-14,  9.47390314e-15],
       [-3.78956126e-14,  9.47390314e-15],
       [ 2.84217094e-14,  0.00000000e+00],
       [ 1.89478063e-14,  0.00000000e+00],
       [ 5.68434189e-14,  9.47390314e-15],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.78956126e-14],
       [ 1.89478063e-14, -3.78956126e-14],
       [ 5.68434189e-14,  3.78956126e-14],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.36847579e-15],
       [ 0.00000000e+00, -7.10542736e-15],
       [-1.89478063e-14,  0.00000000e+00],
       [-3.78956126e-14,  0.00000000e+00],
       [ 3.78956126e-14,  0.00000000e+00],
       [-1.89478063e-14, -1.89478063e-14],
       [-1.